In [17]:
from RPLAN_Toolbox.rplan.floorplan import Floorplan
from RPLAN_Toolbox.rplan.align import align_fp_gt
from RPLAN_Toolbox.rplan.decorate import get_dw
from RPLAN_Toolbox.rplan.measure import compute_tf
from RPLAN_Toolbox.rplan.plot import get_figure,get_axes,plot_category,plot_boundary,plot_graph,plot_fp,plot_tf

In [27]:
DATA_DIR = './raw_data/floorplan_dataset'
file_path = f'{DATA_DIR}/0.png'
fp = Floorplan(file_path)
data = fp.to_dict()

KeyboardInterrupt: 